In [6]:
import sys
sys.path.append("../src")

from data_loader import load_sales_data, load_calendar_data, melt_sales

sales_path = "../data/raw/sales_train_evaluation.csv"
calendar_path = "../data/raw/calendar.csv"

sales = load_sales_data(sales_path)
calendar = load_calendar_data(calendar_path)

sales_long = melt_sales(sales)

sales_long.head()


Loading sales data...
Shape: (30490, 1947)
Loading calendar data...
Shape: (1969, 14)
Melting sales data to long format...
Long format shape: (59181090, 8)


,id,item_id,dept_id,cat_id,store_id,state_id,d,units_sold
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0


In [7]:
sales_merged = sales_long.merge(
    calendar[['d', 'date', 'wm_yr_wk']],
    on='d',
    how='left'
)

print(sales_merged.shape)
sales_merged.head()


(59181090, 10)


,id,item_id,dept_id,cat_id,store_id,state_id,d,units_sold,date,wm_yr_wk
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101


In [8]:
weekly_sales = (
    sales_merged
    .groupby(['item_id', 'store_id', 'wm_yr_wk'], as_index=False)
    ['units_sold']
    .sum()
)

print(weekly_sales.shape)
weekly_sales.head()


(8476220, 4)


,item_id,store_id,wm_yr_wk,units_sold
0,FOODS_1_001,CA_1,11101,10
1,FOODS_1_001,CA_1,11102,6
2,FOODS_1_001,CA_1,11103,10
3,FOODS_1_001,CA_1,11104,13
4,FOODS_1_001,CA_1,11105,15


In [9]:
del sales
del sales_long
del sales_merged
import gc
gc.collect()


0

In [ ]:
weekly_sales.to_csv("../data/processed/weekly_sales.csv", index=False)
